In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
import matplotlib as mpl
import matplotlib.pyplot as plt



import statsmodels.api as sm

# readable figures
pd.set_option('float_format', '{:f}'.format)

In [3]:
df = pd.read_csv('../input/pertanianmultinomial/uas.csv')
df.head()

In [4]:
df.describe()

In [5]:
display(df)

In [6]:
x = df.drop('Category', axis=1)
x.head()

In [7]:
y = df['Category']
y.head()

In [8]:
from sklearn.preprocessing import StandardScaler 
from sklearn import preprocessing
x_scaled = StandardScaler().fit_transform(x)
data_scaled = pd.DataFrame(x_scaled, columns=df.columns.drop(['Category']))

data_scaled

In [9]:
display(df)

In [10]:
trainX, testX, trainY, testY = train_test_split(x, y, test_size = 0.3)

In [11]:
mnlogit_mod = sm.MNLogit(y,sm.add_constant(data_scaled))
mnlogit_fit = mnlogit_mod.fit(method='bfgs')

print (mnlogit_fit.summary())

In [12]:
logistic_regresion = LogisticRegression(multi_class='auto').fit(x,y)
y_pred = logistic_regresion.predict(testX)
y_pred

In [13]:
print('Intercept :', logistic_regresion.intercept_)

In [14]:
print(logistic_regresion.coef_)

In [15]:
prediction_test = logistic_regresion.predict(testX)

In [16]:
print('Accuracy: {:.2f}'.format(accuracy_score(testY, y_pred)))
print('Error rate: {:.2f}'.format(1 - accuracy_score(testY, y_pred)))

In [17]:
confusion_matrix = confusion_matrix(testY, y_pred)
print(confusion_matrix)

In [18]:
plt.matshow(confusion_matrix, cmap=plt.cm.gray)
plt.show()

In [19]:
probability = logistic_regresion.predict_proba(testX)
probability

In [20]:
print(probability.shape[0])
print(testX.shape[0])

In [21]:
#data_result = pd.DataFrame(logistic_regresion.coef_, columns=x.columns)
#data_result['intercept'] = log_reg_result.intercept_
#data_result['class'] = logistic_regresion.classes_

#data_result

In [22]:
data_result = pd.DataFrame(logistic_regresion.predict_proba(testX), columns=logistic_regresion.classes_)
data_result

In [23]:
data_result['sum'] = data_result.sum(axis=1)
data_result

In [24]:
data_result['predicted_class'] = y_pred
data_result

In [25]:
data_result['actual_class'] = testY.to_frame().reset_index().drop(columns='index')
data_result

In [26]:
le = preprocessing.LabelEncoder()

data_result['label_pred'] = le.fit_transform(data_result['predicted_class'])
data_result['label_actual'] = le.fit_transform(data_result['actual_class'])
data_result

In [27]:
targets = data_result['predicted_class']   
integerEncoded = le.fit_transform(targets)
integerMapping=dict(zip(targets,integerEncoded))
integerMapping

In [28]:
targets = data_result['actual_class']   
integerEncoded = le.fit_transform(targets)
integerMapping=dict(zip(targets,integerEncoded))
integerMapping

In [29]:
data_result['check'] = data_result['label_actual'] - data_result['label_pred']
data_result

In [30]:
data_result['correct_prediction?'] = np.where(data_result['check'] == 0, 'True', 'False')
data_result = data_result.drop(['label_pred', 'label_actual', 'check'], axis=1)
data_result

In [31]:
display(data_result)

In [32]:
true_predictions = data_result[(data_result["correct_prediction?"] == 'True')].shape[0]
false_predictions = data_result[(data_result["correct_prediction?"] == 'False')].shape[0]
total = data_result["correct_prediction?"].shape[0]

print('manual calculated Accuracy is:', (true_predictions / total * 100))

In [33]:
wrong_pred = data_result[(data_result["correct_prediction?"] == 'False')]
wrong_pred

In [34]:
x2 = sm.add_constant(data_scaled, prepend = False)

mnlogit_mod = sm.MNLogit(y, x2).fit(method='bfgs')

print (mnlogit_mod.summary())